In [1]:
#imprtant libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import os

In [2]:
#directories
orginal_dir='/home/usama/Desktop/obd2_project'
data_dir='/home/usama/Desktop/obd2_project/OBD II Dataset'

In [3]:
#import csv directory folder
dataset=os.path.join(data_dir,"OBD.csv")

In [4]:
#read csv file
dataframe=pd.read_csv(dataset,delimiter=',')

In [5]:
#set the values
data_set=dataframe.values

In [6]:
#shape of the data set
np.shape(data_set)

(1356, 18)

In [7]:
#split GPS time into anohter format
GPS_data=dataframe['GPS Time'].str.split(' ',n=5,expand=True)
dataframe['Day']=GPS_data[0]
dataframe['Month']=GPS_data[1]
dataframe['Date']=GPS_data[2]
dataframe['Time']=GPS_data[3]
dataframe['Time zone']=GPS_data[4]
dataframe['Year']=GPS_data[5]

In [8]:
#split  time into another format
TIME_data=dataframe['Time'].str.split(':',n=3,expand=True)
dataframe['Hours']=TIME_data[0]
dataframe['Minutes']=TIME_data[1]
dataframe['Seconds']=TIME_data[2]

In [18]:
#find no of columns
dataframe.columns

Index([' Device Time', ' Longitude', ' Latitude', 'GPS Speed (Meters/second)',
       ' Horizontal Dilution of Precision', ' Altitude', ' Bearing', ' G(x)',
       ' G(y)', ' G(z)', ' G(calibrated)', 'Engine Coolant Temperature(°C)',
       'Engine RPM(rpm)', 'Intake Air Temperature(°C)', 'Engine Load(%)',
       'Mass Air Flow Rate(g/s)', 'Throttle Position(Manifold)(%)', 'Day',
       'Month', 'Date', 'Time zone', 'Year', 'Hours', 'Minutes', 'Seconds'],
      dtype='object')

In [9]:
#drop all num value in our dataset
dataframe.dropna(inplace=True)

In [10]:
#drop time column from dataframe
dataframe.drop(columns='Time',inplace=True)

In [11]:
#drop GPS time column from dataframe
dataframe.drop(columns='GPS Time',inplace=True)

In [19]:
#drop Device time column from dataframe
dataframe.drop(columns=" Device Time",inplace=True)

In [20]:
#describe our dataset
dataframe.describe()

,Longitude,Latitude,GPS Speed (Meters/second),Horizontal Dilution of Precision,Altitude,Bearing,G(x),G(y),G(z),G(calibrated),Engine Coolant Temperature(°C),Engine RPM(rpm),Intake Air Temperature(°C),Engine Load(%),Mass Air Flow Rate(g/s),Throttle Position(Manifold)(%)
count,1356.000000,1356.000000,1356.000000,1356.000000,1356.000000,1356.000000,1356.000000,1356.000000,1356.000000,1356.000000,1356.000000,1356.000000,1356.000000,1356.000000,1356.000000,1356.000000
mean,-97.716240,30.536503,13.644352,52.460914,236.735988,211.945133,0.572515,6.510720,6.482762,0.016298,84.085546,1738.828540,31.179204,55.497427,13.425442,22.547574
std,0.040513,0.030855,10.319835,47.350128,47.046773,140.549289,1.528175,2.076501,2.229453,0.071754,5.643929,706.541733,1.066585,26.489267,12.460740,8.887587
min,-97.777058,30.482689,0.000000,11.000000,60.000000,0.000000,-7.692474,-2.073723,-4.262598,-0.305164,56.000000,614.000000,30.000000,16.470589,2.090000,14.901961
25%,-97.762114,30.508071,2.557545,18.000000,217.000000,38.150000,-0.235302,5.532770,5.567860,-0.019881,85.000000,1051.125000,30.000000,32.156864,2.950000,15.686275
50%,-97.693833,30.550567,13.560015,29.000000,229.000000,258.550000,0.600083,6.680282,6.784478,0.014268,86.000000,1809.250000,31.000000,48.235294,4.520000,16.862745
75%,-97.685137,30.557920,22.986395,91.000000,252.000000,336.125000,1.576419,7.715458,7.848844,0.049367,86.000000,2359.250000,32.000000,81.568626,24.502500,27.450981
max,-97.662474,30.574231,32.540222,246.000000,454.000000,359.900000,4.602767,11.798932,19.633602,0.806779,90.000000,4107.000000,36.000000,100.000000,55.950001,66.666664


In [21]:
#show some data
dataframe.head()

,Longitude,Latitude,GPS Speed (Meters/second),Horizontal Dilution of Precision,Altitude,Bearing,G(x),G(y),G(z),G(calibrated),...,Mass Air Flow Rate(g/s),Throttle Position(Manifold)(%),Day,Month,Date,Time zone,Year,Hours,Minutes,Seconds
0,-97.662474,30.483089,2.795157,93,250,211.6,0.350434,9.163701,-2.947818,0.000910,...,33.759998,32.549019,Thu,Nov,02,CDT,2017,15,44,24
1,-97.662476,30.483068,0.000000,98,255,0.0,-6.745979,4.394452,-4.039574,-0.062790,...,37.200001,32.549019,Thu,Nov,02,CDT,2017,15,44,25
2,-97.662478,30.483006,0.000000,105,247,0.0,-0.923281,7.736757,3.870868,-0.094129,...,35.180000,37.647060,Thu,Nov,02,CDT,2017,15,44,26
3,-97.662546,30.482689,0.000000,101,225,0.0,-4.792311,2.172633,16.387754,0.773513,...,38.040001,39.215687,Thu,Nov,02,CDT,2017,15,44,27
4,-97.662546,30.482689,0.000000,101,225,0.0,-0.061368,7.190879,6.686449,0.038575,...,39.349998,38.823528,Thu,Nov,02,CDT,2017,15,44,27


In [52]:
#to save the csvfile
modified=dataframe.to_csv(r'/home/usama/Desktop/obd2_project/OBD II Dataset/Modified_OBDII.csv',index=False)

In [ ]:
modifieda